<a href="https://colab.research.google.com/github/anadcruz/LLM_Aula5/blob/main/aula05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#Instalando o SDK do Google
!pip install -q -U google-generativeai

In [7]:

# Import the Python SDK
import numpy as np
import pandas as pd
import google.generativeai as genai

from google.colab import userdata
api_key = userdata.get("SECRET_KEY")
genai.configure(api_key=api_key)

Listando os modelos disponíveis de Embeddings - serve para o modelo entender melhor o contexto semantico no qual a palavra está inserida através das distancias de tokens em um campo....converte uma img , texto em visão numérica para o computador entender

In [8]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [9]:
#exemplo de embedding:

title = "A proxima geração de IA para desenvolveddores e Google WorkSpace"
sample_text = ("Título : A proxima geração de IA para desenvolveddores e Google WorkSpace"
"\n"
"Artigo completo:\n "
"\n"
"Gemini API & Google AI Studio : uma maneira acessível de explorar e criar protótipos com aplicações de AI generativa")

embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")
print(embeddings)

{'embedding': [0.048768084, -0.050447874, -0.014247892, 0.02274507, 0.066609144, -0.0025272435, -0.019806484, -0.024441335, 0.060331937, 0.05289145, 0.0100548705, 0.0068377787, -0.049858205, -0.017689973, 0.009072699, -0.027856756, 0.018684015, -0.0047453297, -0.039977726, 0.0013145945, 0.01504034, 0.008401764, -0.028813373, -0.07393071, -0.015120381, 0.018331258, 0.023485335, -0.03390706, -0.04269135, 0.027530354, -0.050336216, 0.04938441, -0.034728568, 0.013375253, -0.02147653, -0.045603443, -0.008986809, -0.03446193, -4.24702e-05, -0.009250199, 0.0142119685, -0.085176505, -0.010647742, 0.016590206, -0.011351898, -0.018025557, 0.04494159, 0.049560443, 0.022964377, -0.05720505, 0.03523149, 0.019265488, 0.07030474, -0.040632777, 0.0026728588, -0.013679698, 0.023264047, -0.029746737, 0.023846153, 0.011777145, -0.008614416, 0.026167804, -0.011371536, 0.054708723, 0.028956922, -0.057019174, -0.036397286, 0.0010405973, 0.015423127, 0.03134842, 0.007344838, 0.0024606737, 0.063434035, -0.061

In [10]:
DOCUMENT1 = {
    "title": "Operating the Climate Control System",
    "content": "Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."}
DOCUMENT2 = {
    "title": "Touchscreen",
    "content": "Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the \"Navigation\" icon to get directions to your destination or touch the \"Music\" icon to play your favorite songs."}
DOCUMENT3 = {
    "title": "Shifting Gears",
    "content": "Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [11]:
df = pd.DataFrame(documents)
df.columns = ["Titulo","Conteudo"]
df

,Titulo,Conteudo
0,Operating the Climate Control System,Your Googlecar has a climate control system th...
1,Touchscreen,Your Googlecar has a large touchscreen display...
2,Shifting Gears,Your Googlecar has an automatic transmission. ...


In [12]:
model = "models/embedding-001"



In [13]:
def embed_fn(title,text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [14]:
df["Embeddings"]= df.apply(lambda row: embed_fn(row["Titulo"],row["Conteudo"]),axis=1)
df

,Titulo,Conteudo,Embeddings
0,Operating the Climate Control System,Your Googlecar has a climate control system th...,"[-0.033361107, -0.021217084, -0.049581926, -0...."
1,Touchscreen,Your Googlecar has a large touchscreen display...,"[0.009660736, -0.030662702, -0.017281422, -0.0..."
2,Shifting Gears,Your Googlecar has an automatic transmission. ...,"[-0.04270796, -0.007160868, -0.03242516, -0.02..."


In [15]:
def gerar_e_buscar_consulta(consulta,base,model):
 embedding_da_consulta=genai.embed_content(model=model,
                                 content=consulta,
                                task_type="RETRIEVAL_QUERY")
 produtos_escalares=np.dot(np.stack(df["Embeddings"]), embedding_da_consulta["embedding"])

 indice = np.argmax(produtos_escalares)
 return df.iloc[indice]["Conteudo"]

In [17]:
consulta = "How do I change gears in a Google car?"


trecho = gerar_e_buscar_consulta(consulta,df,model)
print(trecho)

Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions.


In [24]:
generation_config={
    "candidate_count": 1,
    "temperature": 0,
}

In [25]:
prompt = f"Reescreva esse texto de forma mais descontraida , sem adicionar informações que não façam parte do texto: {trecho}"
model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

**Guia de marchas do seu Googlecar:**

* **Estacionado:** Quando você estaciona, é só deixar nessa marcha. As rodas ficam travadas e o carro não mexe.
* **Ré:** Para dar aquela rézinha básica.
* **Neutro:** Quando você para no sinal ou no trânsito. O carro não está engatado e só vai andar se você pisar no acelerador.
* **Dirigir:** Para ir pra frente, é só engatar essa marcha.
* **Baixa:** Para quando a coisa fica escorregadia, como na neve ou na lama.
